<a href="https://colab.research.google.com/github/gauravbyte/DL_Assignment_3/blob/master/DL_ASSIGNMENT_3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df_train = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_test.csv")
df_valid = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_valid.csv")

In [3]:
print(df_train)
print(df_test)
print(df_valid)

       shastragaar शस्त्रागार
0          bindhya   बिन्द्या
1        kirankant   किरणकांत
2      yagyopaveet  यज्ञोपवीत
3          ratania    रटानिया
4       vaganyache  वागण्याचे
...            ...        ...
51194        toned       टोंड
51195   mutanaazaa   मुतनाज़ा
51196    asahmaton    असहमतों
51197    sulgaayin   सुलगायीं
51198  anchuthengu  अंचुतेंगु

[51199 rows x 2 columns]
           thermax      थरमैक्स
0        sikhaaega      सिखाएगा
1            learn         लर्न
2         twitters     ट्विटर्स
3      tirunelveli  तिरुनेलवेली
4     independence  इंडिपेंडेंस
...            ...          ...
4090       saflata       सफ़लता
4091        shbana        शबाना
4092  khaatootolaa     खातूटोला
4093    shivastava     शिवास्तव
4094  preranapuree  प्रेरणापुरी

[4095 rows x 2 columns]
         jaisawal       जयसवाल
0           bajai         बजाई
1       sanghthan        संघठन
2         haiwaan        हैवान
3         nilgiri      नीलगिरि
4       drutgrami  द्रुतग्रामी
...           ...  